In [2]:
import numpy as np
import PIL.Image as image
import os
import matplotlib.pyplot as plt
import cv2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets,models,transforms
import time
import copy

In [4]:
import pkg_resources

# List all installed packages
installed_packages = pkg_resources.working_set

# Filter packages containing "torch" in their name
torch_packages = [pkg for pkg in installed_packages if "torch" in pkg.key.lower()]

# Print the list of torch-related packages
for pkg in torch_packages:
    print(f"{pkg.key}: {pkg.version}")


torchvision: 0.15.2
torch: 2.0.1


In [5]:
torch.__version__

'2.0.1+cpu'

In [6]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    # Use CUDA device
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    # Use CPU
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

# Now you can create and use PyTorch tensors, models, and other functionalities with CUDA support.


CUDA is not available. Using CPU.


In [7]:
import torch.cuda
torch.cuda.is_available()

False

In [8]:
import torch
print("Num GPUs Available: ", torch.cuda.device_count())


Num GPUs Available:  0


In [9]:
torch.__version__

'2.0.1+cpu'

In [10]:
data_dir_train='New Plant Diseases Dataset(Augmented)/train'
data_dir_test='New Plant Diseases Dataset(Augmented)/valid'

In [1]:
import pathlib
data_dir_train=pathlib.Path(data_dir_train)
data_dir_train

NameError: name 'data_dir_train' is not defined

In [12]:
classes = [subdir.name for subdir in data_dir_train.glob('*') if subdir.is_dir()]



In [13]:
len(classes)

38

In [13]:
classes


['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___healthy',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___healthy',
 'Potato___Late_blight',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___healthy',
 'Strawberry___Leaf_scorch',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___healthy',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spid

In [14]:
leaves_labels={}
for index,class_names in enumerate(classes):
    leaves_labels[class_names]=index
    # leaves_labels[class_names]=i

In [15]:
leaves={}
for class_name in classes:
    class_path=data_dir_train/class_name
    leaves[class_name]=list(class_path.glob('*'))

y_train=[]
for plant_name,path in leaves.items():
    for image_path in path:
        y_train.append(leaves_labels[plant_name])
# y_train

In [17]:
data_dir_test=pathlib.Path(data_dir_test)


In [18]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize((244, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

In [19]:
from torchvision.datasets import ImageFolder
train_dataset = ImageFolder(root=data_dir_train, transform=transform)
test_dataset = ImageFolder(root=data_dir_test, transform=transform)

In [20]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=2,shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

In [21]:
from PIL import Image as image

In [22]:
train_dataset[0][0].shape

torch.Size([3, 244, 224])

In [23]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)

Using cache found in C:\Users\niran/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\niran\OneDrive\Desktop\INTERNSHIP\IIT INTERNSHIP\Plant Disease Model\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\niran\OneDrive\Desktop\INTERNSHIP\IIT INTERNSHIP\Plant Disease Model\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
print(model)

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   

In [25]:
model.features.parameters()

<generator object Module.parameters at 0x000001A3F3BB47B0>

In [26]:
torch.__version__

'2.0.1+cpu'

In [29]:
num_classes=len(classes)
# Modify the classifier layers for the new number of classes
for param in model.parameters():
    param.requires_grad=False
# model.features[10].requires_grad=True
# model.features[12].requires_grad=True
model.classifier=nn.Sequential(
    nn.Dropout(p=0.5, inplace=False),
    nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1)),
    nn.ReLU(inplace=True),
    nn.AdaptiveAvgPool2d(output_size=(1, 1))

    )

# Define the device (GPU if available, otherwise CPU)
device = torch.device("cpu")
model.to(device)

# Define your criterion (loss function)
criterion = nn.CrossEntropyLoss()

# Define your optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        try:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        except FileNotFoundError:
            print("The file didnt exist")
            continue
            

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader)}")

# Evaluation
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)


test_accuracy = correct_predictions / total_predictions
print("Test Accuracy: {:.2%}".format(test_accuracy))  


Epoch [1/5], Loss: 0.2555884207932132
Epoch [2/5], Loss: 0.1465032763090223
Epoch [3/5], Loss: 0.1321613697301866
Epoch [4/5], Loss: 0.12370786037610157
Epoch [5/5], Loss: 0.12036520913546059
Test Accuracy: 97.00%


In [31]:
import torchvision.models as models



# Count the number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())

# Calculate the model size in bytes
model_size_bytes = total_params * 4  # Assuming 32-bit float (4 bytes) for each parameter

print("Model size in bytes: ", model_size_bytes)
size_kb = model_size_bytes / 1024
size_mb = size_kb / 1024

print("Model size in KB: ", size_kb)
print("Model size in MB: ", size_mb)


Model size in bytes:  3019672
Model size in KB:  2948.8984375
Model size in MB:  2.8797836303710938


In [28]:
from PIL import Image

image_path = 'New Plant Diseases Dataset(Augmented)/valid/Apple___Apple_scab/0a5e9323-dbad-432d-ac58-d291718345d9___FREC_Scab 3417 (2).JPG'  # Replace with the path to your test image



image = Image.open(image_path)
image = transform(image)
image = image.unsqueeze(0)  # Add a batch dimension

# Make predictions
with torch.no_grad():
    outputs = model(image)

# Assuming your model uses CrossEntropyLoss during training
_, predicted = torch.max(outputs, 1)


predicted_label = classes[predicted.item()]

print(f"Predicted label: {predicted_label}")


Predicted label: Cherry_(including_sour)___healthy
